In [3]:
import geopy
import pandas as pd
from geopy.distance import geodesic
from tqdm import tqdm

In [5]:
# ✅ 1. CSV 파일 불러오기 (파일 경로는 실제 파일에 맞게 변경)
apt_df = pd.read_csv("/Users/angwang-yun/Desktop/Project/coor/90k_uder_2B_최종 copy.csv")   # 아파트 데이터
school_df = pd.read_csv("/Users/angwang-yun/Desktop/Project/coor/좌표변환_1.csv")  # 초등학교 데이터
# ✅ 2. 유니크한 아파트 위도·경도 추출
unique_apts = apt_df[['전체주소', '위도', '경도']].drop_duplicates()

In [6]:
# ✅ NaN 값 제거
unique_apts = unique_apts.dropna(subset=['위도', '경도'])
school_df = school_df.dropna(subset=['위도', '경도'])

In [7]:

# ✅ 3. 거리 계산 함수 (미터 단위)
def find_nearest_school(lat, lon, school_df):
    min_distance = float('inf')
    nearest_school = None
    
    # 모든 초등학교와 거리 계산
    for _, row in school_df.iterrows():
        school_lat, school_lon = row['위도'], row['경도']
        distance = geodesic((lat, lon), (school_lat, school_lon)).meters
        
        # 최소 거리 업데이트
        if distance < min_distance:
            min_distance = distance
            nearest_school = row['학교명']  # 초등학교 이름 (컬럼명 확인 필요)
    
    return nearest_school, min_distance

# ✅ 4. 최근접 초등학교 찾기 (tqdm으로 진행률 표시)
nearest_school_list = []
for _, row in tqdm(unique_apts.iterrows(), total=len(unique_apts)):
    school, distance = find_nearest_school(row['위도'], row['경도'], school_df)
    nearest_school_list.append((row['전체주소'], school, distance))

# ✅ 5. 데이터프레임 변환
nearest_school_df = pd.DataFrame(nearest_school_list, columns=['전체주소', '최근접 초등학교', '초등학교 거리(미터)'])

# ✅ 6. 원본 아파트 데이터와 병합
apt_df = apt_df.merge(nearest_school_df, on='전체주소', how='left')

# ✅ 7. CSV로 저장 (필요하면)
apt_df.to_csv("apartment_with_nearest_schools.csv", index=False)

print("✅ 거리 계산 완료! 파일 저장 완료.")

100%|██████████| 5627/5627 [04:40<00:00, 20.06it/s]


✅ 거리 계산 완료! 파일 저장 완료.


In [8]:
apt_df.head()

,단지명,주택유형,전체주소,계약날짜,계약구분보완(범주),계약분기,건축년도,계약연도,계약개월수,층,...,구_단지별_이상치만원(2진),단지클러스터,보증금구간,위도,경도,가까운_지하철역,지하철_거리(m),상권_거리(m),최근접 초등학교,초등학교 거리(미터)
0,대성유니드,아파트,서울특별시 성동구 마장로37길 7,2019-04-30,신규,2,2004,2019,24,18,...,0,4,mid,37.566939,127.038685,마장,386.605843,89439.523820,서울동명초등학교,342.200105
1,행당한진타운,아파트,서울특별시 성동구 행당로 82,2019-04-30,신규,2,2000,2019,24,12,...,0,4,mid,37.554809,127.027400,행당,331.830814,88102.788466,서울금북초등학교,451.914304
2,대림e-편한세상,아파트,서울특별시 성동구 행당로 79,2019-04-30,신규,2,2000,2019,24,2,...,0,4,mid,37.558205,127.027507,행당,201.533731,88479.568599,서울금북초등학교,357.832334
3,경희궁자이(2단지),아파트,서울특별시 종로구 송월길 99,2019-04-30,신규,2,2017,2019,24,8,...,0,2,high,37.570861,126.962352,독립문,566.332822,90155.342224,서울금화초등학교,231.623284
4,경희궁자이(2단지),아파트,서울특별시 종로구 송월길 99,2019-04-30,신규,2,2017,2019,24,6,...,0,2,high,37.570861,126.962352,독립문,566.332822,90155.342224,서울금화초등학교,231.623284
